# Glassdoor Reviews for Data Engineers
### Data Engineering Capstone Project




![title](images/data_engineer_title.png)


#### Project Summary
This project aims to answer the perception of current and past employees regarding Data Engineering roles in their companies. It initially consists of two data sources: one related to the reviews from the employees obtained from Glassdoor and the other dataset contains publicly available information regarding companies (such as the country of origin, the sector and industry)



The project follows these steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 

The project collects data from Glassdoor reviews in a specific format, and also data related to companies (for more analytical aggregations) in order to understand the perception of current and past employees regarding Data Engineering roles in their companies.

The end solution proposed is a pipeline build upon Apache Airflow for ingesting data from Amazon S3 into Redshift, running transformations on Redshift and saving them into a proposed Star Schema.

The end cases proposed are the following:
- To serve any aspiring or current Data Engineer to get insights with summarized data regarding the field and companies.
- To keep track of perception of the field over the years


#### Describe and Gather Data 

The project consists initially of 2 data sources:

* Glassdoor Reviews: Data scraped from Glassdoor regarding the perception of employees on their companies. Freely avaliable on Kaggle (https://www.kaggle.com/datasets/davidgauthier/glassdoor-job-reviews)
* Top companies data: Data obtained from Kaggle about the Forbes Top 2000 companies worlwide (https://www.kaggle.com/datasets/ash316/forbes-top-2000-companies) [Taking "Forbes Top2000 2017.csv" into account as it's the latest information available in the dataset]

Both files use the CSV format. Though this project uses these files as a starting point, the main idea consists on having a separate process that scrapes Glassdoor reviews daily and inserts them into the corresponding bucket for having better insights over time.


### Step 2: Explore and Assess the Data
#### Explore the Data

For the Glassdoor reviews dataset, we could identify the following data quality issues:
* Companies name (firm on the dataset) have a hyphen character instead of a space character for separation.
* Companies name ending with <b>'s</b> have a hypen before the <b>s</b> character
* Job titles have extra spaces at the beginning of the string

For the companies dataset, the data was pretty clean, though some columns contained missing values


Please refer **inspection_notebook.ipynb**


#### Cleaning Steps

Since the transformation happens on Redshift, most the cleaning steps will be done using SQL

For the Glassdoor reviews dataset:

* The hyphens for the companies names are being handled using the REPLACE function, also for removing leading and trailing blanks the TRIM function is used
* For the job titles, the INITCAP function is used to capitalize the first letter of each job

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

The proposed data model consists of the following tables:

* **Fact tables:**

    - reviews (review_id, start_time, company_name, employee_role, number_positive_reviews, number_negative_reviews, number_total_reviews)

* **Dimension tables:**
    - companies (name, country, sector, industry)
    - employee_roles (name, status)
    - time (start_time, day, week, month, year, weekday)

![title](images/logical_modeling_.png)

The reasoning for using fact and dimension tables is that the reviews fact table in combination with the other dimension tables allow us to directly inspect or make aggregations regarding which companies had certain amount of reviews and the perception of employees based on the reviews.


#### 3.2 Mapping Out Data Pipelines

The pipeline consists on the following steps:

* Store the data into Amazon S3 to the corresponding key
* Stage the data from S3 to Redshift
* Perform cleaning and necessary transformations for storing it in the corresponding tables in the Star Schema
* Do quality checks on the data

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

Please refer the dags and plugins folder since it's an Airflow pipeline.
The following pictures show the execution of the DAG and results in Amazon Redshift

![title](images/dag_execution.png)

![title](images/redshift_reviews.png)

As an example of an analytical query, the following  query is proposed to check the companies with best average for Data Engineers in 2021

![title](images/example_query.png)

#### 4.2 Data Quality Checks

The quality checks performed mainly check the nullity on the reviews fact table

* Check the number of positive reviews are not null
* Check the number of negative reviews are not null
* Check the number of total reviews are greater than zero
* Check the dimensions are not null on the fact table

#### 4.3 Data dictionary 

The following data dictionary expressed each of the fields for the data modelling:
![title](images/data_dictionary.png)


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project
    - The initial dataset is not that big (around 400MB) and can fit into most of modern day's computer. Apache Airflow allows us to schedule programmatically any kind of job and Redshift helps up maintain an informed schema while providing scaling capabilities

* Propose how often the data should be updated and why.

    * Depending on how frenquently you would like to inspect into Data Engineering jobs, you could feed data into the corresponding S3 buckets and generate the report. A weekly schedule should be enough for checking how the market and perception regarding certain companies is changing

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     - We would need to check if the performance provided by our Redshift instance is enough or consider upgrading it.
     - Since Airflow is just a scheduler, we would not need to deal with the workers.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     - I would need to set a scheduler interval for the pipeline to run at 5am with an SLA of 1 hour. In case this fails, it would trigger an alert and give us enough time to troubleshoot
 * The database needed to be accessed by 100+ people.
     - Redshift gives us enough capabilities to handle that amount of load since it's considered a MPP database